# Import Libraries

In [76]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model

import json


In [78]:
NUM_HEROES = 123
NUM_FEATURES = NUM_HEROES * 2
ENV_PATH = 'model.h5'

class NNPredictor:
    def __init__(self, env_path = ENV_PATH):
        self.model = tf.keras.models.load_model(env_path)

    def transform(self, my_team, their_team):
        X = np.zeros((1,NUM_FEATURES))
        for hero_id in my_team:
            X[0][(hero_id - 1)] = 1
        for hero_id in their_team:
            X[0][(hero_id - 1 + NUM_HEROES)] = 1
        return X

    def recommend(self, my_team, their_team, hero_candidates):
        team_possibilities = [(candidate, my_team + [candidate]) for candidate in hero_candidates]

        prob_candidate_pairs = []
        for candidate, team in team_possibilities:
            query = self.transform(team, their_team)
            prob = self.score(query)
            prob_candidate_pairs.append((prob, candidate))
        prob_candidate_pairs = sorted(prob_candidate_pairs, reverse = True)[0:5 - len(my_team)]
        return prob_candidate_pairs

    def score(self,query):
        rad_prob = self.model.predict(query)[0][0]
        return rad_prob

    def predict(self, dream_team, their_team):
        '''Returns the probability of the dream_team winning against their_team.'''
        dream_team_query = self.transform(dream_team, their_team)
        return self.score(dream_team_query)
            #return self.model.predict_proba(dream_team_query)[0][1]

In [79]:
with open('heroes.json', 'r') as fp:
    heroes = json.load(fp)
    hero_ids = [hero['id'] for hero in heroes]

def get_hero_human_readable(hero_id):
    for hero in heroes:
        if hero['id'] == hero_id:
            return hero['localized_name']
    return 'Unknown hero: %d' % hero_id


In [88]:
def main():
    my_team = [1,5,28]
    their_team = [22, 23, 50, 60]
    print (f'My Team: {[get_hero_human_readable(hero_id) for hero_id in my_team]}')
    print (f'Their Team:{[get_hero_human_readable(hero_id) for hero_id in their_team]}')
    print ('Recommend:')

    engine = Engine(NNPredictor())
    recommendations = engine.recommend(my_team, their_team)
    print(f'{[(prob, get_hero_human_readable(hero))for prob, hero in recommendations]}')

In [89]:
class Engine:

    def __init__(self, algorithm):
        self.algorithm = algorithm

    def get_candidates(self, my_team, their_team):
        '''Return a list of hero_IDs to consider for recommending'''
        ids = [i for i in hero_ids if i not in my_team and i not in their_team and i not in [
            24, 115, 116, 117, 118, 122, 124, 125, 127, 130, 131, 132, 133, 134]]
        return ids

    def recommend(self, my_team, their_team, human_readable = False):
        '''Return a list of (hero, probability of winning with hero_added) recommend to complete my_team'''

        assert len(my_team) <= 5
        assert len(their_team) <= 5

        hero_candidates = self.get_candidates(my_team, their_team)
        return self.algorithm.recommend(my_team, their_team, hero_candidates)

    def predict(self, my_team, their_team):
        '''Return the probability of the dream_team winning against their team'''
        return self.algorithm.predict(dream_team, their_team)



In [90]:
if __name__ == '__main__':
    main()

My Team: ['Anti-Mage', 'Crystal Maiden', 'Slardar']
Their Team:['Zeus', 'Kunkka', 'Dazzle', 'Night Stalker']
Recommend:
1/1 [==============================] - 0s 16ms/step
[(0.99210185, 'Silencer'), (0.98835564, 'Phantom Assassin')]
